<!-- TABS -->
# Multimodal vector search

The first step in any SuperDuperDB application is to connect to your data-backend with SuperDuperDB:

<!-- TABS -->
## Connect to SuperDuperDB

In [ ]:
# <tab: MongoDB>
from superduperdb import superduper

db = superduper('mongodb://localhost:27017/documents')

In [ ]:
# <tab: SQLite>
from superduperdb import superduper

db = superduper('sqlite://my_db.db')

Once you have done that you are ready to define your datatype(s) which you would like to "search".

<!-- TABS -->
## Create datatype

In [ ]:
# <tab: Audio>
...

In [ ]:
# <tab: Video>
...

<!-- TABS -->
## Insert data

In order to create data, we need create a `Schema` for encoding our special `Datatype` column(s) in the databackend.

Here's some sample data to work with:

In [ ]:
# <tab: Text>
!curl -O https://jupyter-sessions.s3.us-east-2.amazonaws.com/text.json

import json
with open('text.json') as f:
    data = json.load(f)

In [ ]:
# <tab: Images>
!curl -O https://jupyter-sessions.s3.us-east-2.amazonaws.com/images.zip
!unzip images.zip

import os
data = [{'image': f'file://image/{file}'} for file in os.listdir('./images')]

In [ ]:
# <tab: Audio>
!curl -O https://jupyter-sessions.s3.us-east-2.amazonaws.com/audio.zip
!unzip audio.zip

import os
data = [{'audio': f'file://audio/{file}'} for file in os.listdir('./audio')]

The next code-block is only necessary if you're working with a custom `DataType`:

In [ ]:
from superduperdb import Schema, Document

schema = Schema(
    'my_schema',
    fields={
        'my_key': dt
    }
)

data = [
    Document({'my_key': item}) for item in data
]

In [ ]:
# <tab: MongoDB>
from superduperdb.backends.mongodb import Collection

collection = Collection('documents')

db.execute(collection.insert_many(data))

In [ ]:
# <tab: SQL>
from superduperdb.backends.ibis import Table

table = Table(
    'my_table',
    schema=schema,
)

db.add(table)
db.execute(table.insert(data))

<!-- TABS -->
## Build multimodal embedding models

In [ ]:
# <tab: Text>

...

In [ ]:
# <tab: Image>

...

In [ ]:
# <tab: Text-2-Image>

...

<!-- TABS -->
## Perform a vector search

- `item` is the item which is to be encoded
- `dt` is the `DataType` instance to apply

In [ ]:
from superduperdb import Document

item = Document({'my_key': dt(item)})

Once we have this search target, we can execute a search as follows:

In [ ]:
# <tab: MongoDB>
from superduperdb.backends.mongodb import Collection

collection = Collection('documents')

select = collection.find().like(item)

In [ ]:
# <tab: SQL>

# Table was created earlier, before preparing vector-search
table = db.load('table', 'documents')

select = table.like(item)

In [ ]:
results = db.execute(select)